In [2]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
import gc
from tqdm import tqdm
from IPython.display import FileLink
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

## Дважды отобранные фичи с помощью Boruta

In [3]:
features = ['id','target','feature_4', 'feature_5', 'feature_8', 'feature_9', 'feature_11', 'feature_12', 'feature_14', 'feature_16', 'feature_18', 'feature_19', 'feature_29', 'feature_31', 'feature_34', 'feature_35', 'feature_36', 'feature_39', 'feature_42', 'feature_44', 'feature_45', 'feature_47', 'feature_48', 'feature_50', 'feature_51', 'feature_53', 'feature_56', 'feature_57', 'feature_60', 'feature_61', 'feature_62', 'feature_64', 'feature_68', 'feature_73', 'feature_74', 'feature_76', 'feature_80', 'feature_81', 'feature_82', 'feature_83', 'feature_84', 'feature_86', 'feature_87', 'feature_88', 'feature_92', 'feature_97', 'feature_98', 'feature_100', 'feature_101', 'feature_103', 'feature_104', 'feature_105', 'feature_106', 'feature_114', 'feature_117', 'feature_118', 'feature_119', 'feature_121', 'feature_122', 'feature_125', 'feature_126', 'feature_127', 'feature_131', 'feature_132', 'feature_133', 'feature_137', 'feature_138', 'feature_139', 'feature_141', 'feature_143', 'feature_144', 'feature_147', 'feature_151', 'feature_152', 'feature_153', 'feature_154', 'feature_156', 'feature_157', 'feature_158', 'feature_162', 'feature_164', 'feature_166', 'feature_167', 'feature_168', 'feature_170', 'feature_172', 'feature_173', 'feature_175', 'feature_178', 'feature_179', 'feature_186', 'feature_189', 'feature_191', 'feature_194', 'feature_195', 'feature_197', 'feature_198', 'feature_199', 'feature_200', 'feature_201', 'feature_202', 'feature_203', 'feature_204', 'feature_205', 'feature_206', 'feature_208', 'feature_209', 'feature_212', 'feature_214', 'feature_218', 'feature_221', 'feature_222', 'feature_223', 'feature_224', 'feature_225', 'feature_230', 'feature_231', 'feature_232', 'feature_234', 'feature_238', 'feature_239', 'feature_242', 'feature_244', 'feature_246', 'feature_247', 'feature_248', 'feature_251', 'feature_253', 'feature_254', 'feature_255', 'feature_259', 'feature_262', 'feature_264', 'feature_265', 'feature_266', 'feature_270', 'feature_271', 'feature_273', 'feature_275', 'feature_280', 'feature_283', 'feature_285', 'feature_296', 'feature_297', 'feature_303', 'feature_305', 'feature_307', 'feature_308', 'feature_311', 'feature_314', 'feature_315', 'feature_318', 'feature_321', 'feature_323', 'feature_325', 'feature_331', 'feature_332', 'feature_334', 'feature_335', 'feature_336', 'feature_340', 'feature_341', 'feature_342', 'feature_343', 'feature_345', 'feature_347', 'feature_349', 'feature_353', 'feature_356', 'feature_359', 'feature_360', 'feature_361', 'feature_364', 'feature_365', 'feature_366', 'feature_369', 'feature_371', 'feature_373', 'feature_378', 'feature_379', 'feature_381', 'feature_382', 'feature_383', 'feature_386', 'feature_388', 'feature_390', 'feature_392', 'feature_395', 'feature_404', 'feature_406', 'feature_409', 'feature_410', 'feature_412', 'feature_421', 'feature_423', 'feature_424', 'feature_425', 'feature_428', 'feature_429', 'feature_433', 'feature_434', 'feature_435', 'feature_439', 'feature_443', 'feature_446', 'feature_449', 'feature_450', 'feature_451', 'feature_452', 'feature_454', 'feature_456', 'feature_458', 'feature_460', 'feature_461', 'feature_462', 'feature_467', 'feature_470', 'feature_480', 'feature_481', 'feature_482', 'feature_490', 'feature_491', 'feature_495', 'feature_496']

In [4]:
test_features = ['id','feature_4', 'feature_5', 'feature_8', 'feature_9', 'feature_11', 'feature_12', 'feature_14', 'feature_16', 'feature_18', 'feature_19', 'feature_29', 'feature_31', 'feature_34', 'feature_35', 'feature_36', 'feature_39', 'feature_42', 'feature_44', 'feature_45', 'feature_47', 'feature_48', 'feature_50', 'feature_51', 'feature_53', 'feature_56', 'feature_57', 'feature_60', 'feature_61', 'feature_62', 'feature_64', 'feature_68', 'feature_73', 'feature_74', 'feature_76', 'feature_80', 'feature_81', 'feature_82', 'feature_83', 'feature_84', 'feature_86', 'feature_87', 'feature_88', 'feature_92', 'feature_97', 'feature_98', 'feature_100', 'feature_101', 'feature_103', 'feature_104', 'feature_105', 'feature_106', 'feature_114', 'feature_117', 'feature_118', 'feature_119', 'feature_121', 'feature_122', 'feature_125', 'feature_126', 'feature_127', 'feature_131', 'feature_132', 'feature_133', 'feature_137', 'feature_138', 'feature_139', 'feature_141', 'feature_143', 'feature_144', 'feature_147', 'feature_151', 'feature_152', 'feature_153', 'feature_154', 'feature_156', 'feature_157', 'feature_158', 'feature_162', 'feature_164', 'feature_166', 'feature_167', 'feature_168', 'feature_170', 'feature_172', 'feature_173', 'feature_175', 'feature_178', 'feature_179', 'feature_186', 'feature_189', 'feature_191', 'feature_194', 'feature_195', 'feature_197', 'feature_198', 'feature_199', 'feature_200', 'feature_201', 'feature_202', 'feature_203', 'feature_204', 'feature_205', 'feature_206', 'feature_208', 'feature_209', 'feature_212', 'feature_214', 'feature_218', 'feature_221', 'feature_222', 'feature_223', 'feature_224', 'feature_225', 'feature_230', 'feature_231', 'feature_232', 'feature_234', 'feature_238', 'feature_239', 'feature_242', 'feature_244', 'feature_246', 'feature_247', 'feature_248', 'feature_251', 'feature_253', 'feature_254', 'feature_255', 'feature_259', 'feature_262', 'feature_264', 'feature_265', 'feature_266', 'feature_270', 'feature_271', 'feature_273', 'feature_275', 'feature_280', 'feature_283', 'feature_285', 'feature_296', 'feature_297', 'feature_303', 'feature_305', 'feature_307', 'feature_308', 'feature_311', 'feature_314', 'feature_315', 'feature_318', 'feature_321', 'feature_323', 'feature_325', 'feature_331', 'feature_332', 'feature_334', 'feature_335', 'feature_336', 'feature_340', 'feature_341', 'feature_342', 'feature_343', 'feature_345', 'feature_347', 'feature_349', 'feature_353', 'feature_356', 'feature_359', 'feature_360', 'feature_361', 'feature_364', 'feature_365', 'feature_366', 'feature_369', 'feature_371', 'feature_373', 'feature_378', 'feature_379', 'feature_381', 'feature_382', 'feature_383', 'feature_386', 'feature_388', 'feature_390', 'feature_392', 'feature_395', 'feature_404', 'feature_406', 'feature_409', 'feature_410', 'feature_412', 'feature_421', 'feature_423', 'feature_424', 'feature_425', 'feature_428', 'feature_429', 'feature_433', 'feature_434', 'feature_435', 'feature_439', 'feature_443', 'feature_446', 'feature_449', 'feature_450', 'feature_451', 'feature_452', 'feature_454', 'feature_456', 'feature_458', 'feature_460', 'feature_461', 'feature_462', 'feature_467', 'feature_470', 'feature_480', 'feature_481', 'feature_482', 'feature_490', 'feature_491', 'feature_495', 'feature_496']

In [5]:
df_1 = pd.read_csv('/kaggle/input/first-five-train/Копия train_1.csv', usecols=features)
df_2 = pd.read_csv('/kaggle/input/first-five-train/Копия train_2.csv', usecols=features)
df_3 = pd.read_csv('/kaggle/input/first-five-train/Копия train_3.csv', usecols=features)
df_4 = pd.read_csv('/kaggle/input/first-five-train/Копия train_4.csv', usecols=features)
df_5 = pd.read_csv('/kaggle/input/first-five-train/Копия train_5.csv', usecols=features)
df_6 = pd.read_csv('/kaggle/input/second-train/Копия train_6.csv', usecols=features)
df_7 =pd.read_csv('/kaggle/input/second-train/Копия train_7.csv', usecols=features)
df_8 = pd.read_csv('/kaggle/input/second-train/Копия train_8.csv', usecols=features)
df_9 = pd.read_csv('/kaggle/input/second-train/Копия train_9.csv', usecols=features)
df_10 = pd.read_csv('/kaggle/input/second-train/Копия train_10.csv', usecols=features)

In [6]:
dfs = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10]

for df in dfs:
    for col in df.select_dtypes(include=['int']):
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float']):
        df[col] = pd.to_numeric(df[col], downcast='float')
df = pd.concat(dfs, ignore_index=True)
print(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4490468 entries, 0 to 4490467
Columns: 224 entries, target to feature_496
dtypes: float32(222), int32(1), int8(1)
memory usage: 3.7 GB
None


In [7]:
import gc
del df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10
gc.collect()

0

## Отобранные фичи c помощью Boruta после генерации фичей

In [8]:
generated = ['id','target','feature_4', 'feature_5', 'feature_8', 'feature_9', 'feature_11', 'feature_12', 'feature_14', 'feature_16', 'feature_18', 'feature_29', 'feature_31', 'feature_34', 'feature_42', 'feature_44', 'feature_45', 'feature_47', 'feature_48', 'feature_50', 'feature_51', 'feature_53', 'feature_56', 'feature_57', 'feature_60', 'feature_62', 'feature_64', 'feature_68', 'feature_76', 'feature_80', 'feature_81', 'feature_82', 'feature_83', 'feature_84', 'feature_86', 'feature_87', 'feature_92', 'feature_97', 'feature_98', 'feature_100', 'feature_103', 'feature_105', 'feature_106', 'feature_114', 'feature_117', 'feature_118', 'feature_119', 'feature_121', 'feature_127', 'feature_131', 'feature_132', 'feature_133', 'feature_137', 'feature_138', 'feature_139', 'feature_141', 'feature_143', 'feature_144', 'feature_147', 'feature_151', 'feature_152', 'feature_153', 'feature_154', 'feature_157', 'feature_158', 'feature_164', 'feature_166', 'feature_167', 'feature_168', 'feature_170', 'feature_172', 'feature_173', 'feature_178', 'feature_186', 'feature_189', 'feature_191', 'feature_194', 'feature_195', 'feature_197', 'feature_198', 'feature_199', 'feature_200', 'feature_201', 'feature_202', 'feature_205', 'feature_206', 'feature_208', 'feature_209', 'feature_212', 'feature_214', 'feature_218', 'feature_222', 'feature_223', 'feature_224', 'feature_230', 'feature_231', 'feature_234', 'feature_238', 'feature_239', 'feature_242', 'feature_244', 'feature_246', 'feature_247', 'feature_251', 'feature_253', 'feature_262', 'feature_264', 'feature_270', 'feature_273', 'feature_280', 'feature_283', 'feature_285', 'feature_296', 'feature_297', 'feature_305', 'feature_307', 'feature_308', 'feature_311', 'feature_314', 'feature_315', 'feature_318', 'feature_323', 'feature_325', 'feature_331', 'feature_332', 'feature_334', 'feature_335', 'feature_336', 'feature_340', 'feature_341', 'feature_342', 'feature_343', 'feature_345', 'feature_347', 'feature_349', 'feature_353', 'feature_356', 'feature_359', 'feature_361', 'feature_364', 'feature_365', 'feature_366', 'feature_369', 'feature_371', 'feature_378', 'feature_379', 'feature_381', 'feature_382', 'feature_383', 'feature_386', 'feature_388', 'feature_390', 'feature_392', 'feature_395', 'feature_406', 'feature_410', 'feature_421', 'feature_423', 'feature_424', 'feature_425', 'feature_428', 'feature_429', 'feature_434', 'feature_435', 'feature_443', 'feature_446', 'feature_449', 'feature_451', 'feature_452', 'feature_454', 'feature_456', 'feature_458', 'feature_460', 'feature_461', 'feature_462', 'feature_467', 'feature_470', 'feature_480', 'feature_481', 'feature_482', 'feature_491', 'feature_495', 'feature_pca0', 'feature_pca1', 'feature_pca2', 'feature_pca3', 'feature_pca4', 'feature_pca5', 'feature_pca6', 'feature_pca7', 'feature_pca9', 'feature_pca10', 'feature_pca11', 'feature_pca13', 'feature_pca14', 'feature_pca15', 'feature_pca16', 'feature_pca17', 'feature_pca18', 'feature_pca19', 'feature_pca20', 'feature_pca21', 'feature_pca22', 'feature_pca23', 'feature_pca24', 'column_0_mul_1', 'column_0_div_1', 'column_0_div_2', 'column_0_mul_3', 'column_0_div_3', 'column_0_mul_4', 'column_0_div_4', 'column_0_div_5', 'column_0_mul_6', 'column_0_div_6', 'column_0_mul_7', 'column_0_div_7', 'column_0_mul_8', 'column_0_div_8', 'column_0_mul_10', 'column_0_mul_11', 'column_0_div_11', 'column_0_div_14', 'column_0_div_16', 'column_0_div_17', 'column_0_mul_18', 'column_0_mul_20', 'column_0_div_20', 'column_0_mul_21', 'column_0_div_21', 'column_0_mul_23', 'column_0_div_23', 'column_0_mul_24', 'column_0_div_24', 'column_1_mul_3', 'column_1_div_3', 'column_1_mul_4', 'column_1_div_4', 'column_1_mul_5', 'column_1_div_5', 'column_1_mul_6', 'column_1_div_6', 'column_1_mul_8', 'column_1_div_8', 'column_1_mul_11', 'column_1_div_11', 'column_1_mul_16', 'column_1_div_16', 'column_1_mul_19', 'column_1_div_19', 'column_1_mul_20', 'column_1_div_20', 'column_1_mul_21', 'column_1_div_21', 'column_1_mul_23', 'column_1_div_23', 'column_1_mul_24', 'column_1_div_24', 'column_2_mul_3', 'column_2_div_3', 'column_2_mul_4', 'column_2_mul_5', 'column_2_mul_6', 'column_2_mul_8', 'column_2_div_8', 'column_2_mul_11', 'column_2_div_11', 'column_2_mul_16', 'column_2_div_16', 'column_2_mul_17', 'column_2_div_17', 'column_2_mul_19', 'column_2_mul_20', 'column_2_mul_21', 'column_2_mul_23', 'column_2_div_23', 'column_2_mul_24', 'column_3_mul_4', 'column_3_div_4', 'column_3_mul_5', 'column_3_div_5', 'column_3_mul_6', 'column_3_div_6', 'column_3_mul_7', 'column_3_div_7', 'column_3_mul_8', 'column_3_div_8', 'column_3_mul_10', 'column_3_mul_11', 'column_3_div_11', 'column_3_mul_12', 'column_3_mul_13', 'column_3_mul_14', 'column_3_mul_16', 'column_3_div_16', 'column_3_mul_17', 'column_3_div_17', 'column_3_mul_19', 'column_3_div_19', 'column_3_mul_20', 'column_3_div_20', 'column_3_mul_21', 'column_3_div_21', 'column_3_mul_22', 'column_3_div_22', 'column_3_mul_23', 'column_3_div_23', 'column_3_mul_24', 'column_3_div_24', 'column_4_mul_5', 'column_4_div_5', 'column_4_mul_6', 'column_4_div_6', 'column_4_mul_7', 'column_4_mul_8', 'column_4_div_8', 'column_4_mul_10', 'column_4_mul_11', 'column_4_div_11', 'column_4_mul_12', 'column_4_mul_16', 'column_4_div_16', 'column_4_mul_17', 'column_4_mul_19', 'column_4_mul_20', 'column_4_div_20', 'column_4_mul_21', 'column_4_div_21', 'column_4_mul_23', 'column_4_div_23', 'column_4_mul_24', 'column_5_mul_6', 'column_5_mul_7', 'column_5_mul_8', 'column_5_div_8', 'column_5_mul_10', 'column_5_mul_11', 'column_5_div_11', 'column_5_mul_12', 'column_5_mul_14', 'column_5_mul_16', 'column_5_div_16', 'column_5_mul_17', 'column_5_div_17', 'column_5_mul_19', 'column_5_mul_20', 'column_5_div_20', 'column_5_mul_21', 'column_5_div_21', 'column_5_mul_23', 'column_5_div_23', 'column_5_mul_24', 'column_5_div_24', 'column_6_mul_7', 'column_6_mul_8', 'column_6_div_8', 'column_6_mul_10', 'column_6_mul_11', 'column_6_div_11', 'column_6_mul_12', 'column_6_mul_14', 'column_6_mul_16', 'column_6_div_16', 'column_6_mul_17', 'column_6_mul_19', 'column_6_mul_20', 'column_6_div_20', 'column_6_mul_21', 'column_6_div_21', 'column_6_mul_22', 'column_6_mul_23', 'column_6_div_23', 'column_6_mul_24', 'column_7_mul_8', 'column_7_div_8', 'column_7_mul_11', 'column_7_div_11', 'column_7_mul_16', 'column_7_div_16', 'column_7_mul_19', 'column_7_div_19', 'column_7_mul_20', 'column_7_mul_21', 'column_7_mul_23', 'column_7_div_23', 'column_7_mul_24', 'column_8_mul_10', 'column_8_mul_11', 'column_8_div_11', 'column_8_mul_12', 'column_8_mul_13', 'column_8_div_13', 'column_8_mul_14', 'column_8_mul_16', 'column_8_div_16', 'column_8_mul_17', 'column_8_div_17', 'column_8_mul_19', 'column_8_div_19', 'column_8_mul_20', 'column_8_div_20', 'column_8_mul_21', 'column_8_div_21', 'column_8_mul_23', 'column_8_div_23', 'column_8_mul_24', 'column_8_div_24', 'column_9_mul_11', 'column_10_mul_11', 'column_10_div_11', 'column_10_mul_16', 'column_10_mul_23', 'column_10_mul_24', 'column_11_mul_12', 'column_11_div_12', 'column_11_mul_13', 'column_11_div_13', 'column_11_mul_14', 'column_11_div_14', 'column_11_mul_15', 'column_11_div_15', 'column_11_mul_16', 'column_11_div_16', 'column_11_mul_17', 'column_11_div_17', 'column_11_mul_18', 'column_11_div_18', 'column_11_mul_19', 'column_11_div_19', 'column_11_mul_20', 'column_11_div_20', 'column_11_mul_21', 'column_11_div_21', 'column_11_mul_22', 'column_11_div_22', 'column_11_mul_23', 'column_11_div_23', 'column_11_mul_24', 'column_11_div_24', 'column_12_mul_16', 'column_12_div_16', 'column_12_mul_23', 'column_12_mul_24', 'column_13_mul_16', 'column_13_div_16', 'column_13_mul_20', 'column_13_mul_21', 'column_13_mul_23', 'column_14_mul_16', 'column_14_mul_23', 'column_15_div_23', 'column_16_mul_17', 'column_16_div_17', 'column_16_mul_19', 'column_16_div_19', 'column_16_mul_20', 'column_16_div_20', 'column_16_mul_21', 'column_16_div_21', 'column_16_mul_22', 'column_16_mul_23', 'column_16_div_23', 'column_16_mul_24', 'column_16_div_24', 'column_17_mul_19', 'column_17_mul_20', 'column_17_mul_23', 'column_17_div_23', 'column_17_mul_24', 'column_17_div_24', 'column_18_div_23', 'column_19_mul_20', 'column_19_div_20', 'column_19_mul_21', 'column_19_div_21', 'column_19_mul_23', 'column_19_div_23', 'column_19_mul_24', 'column_19_div_24', 'column_20_mul_21', 'column_20_div_21', 'column_20_mul_22', 'column_20_div_22', 'column_20_mul_23', 'column_20_div_23', 'column_20_mul_24', 'column_20_div_24', 'column_21_mul_22', 'column_21_div_22', 'column_21_mul_23', 'column_21_div_23', 'column_21_mul_24', 'column_21_div_24', 'column_22_mul_23', 'column_23_mul_24', 'column_23_div_24']

In [ ]:
test_generate = ['id','feature_4', 'feature_5', 'feature_8', 'feature_9', 'feature_11', 'feature_12', 'feature_14', 'feature_16', 'feature_18', 'feature_29', 'feature_31', 'feature_34', 'feature_42', 'feature_44', 'feature_45', 'feature_47', 'feature_48', 'feature_50', 'feature_51', 'feature_53', 'feature_56', 'feature_57', 'feature_60', 'feature_62', 'feature_64', 'feature_68', 'feature_76', 'feature_80', 'feature_81', 'feature_82', 'feature_83', 'feature_84', 'feature_86', 'feature_87', 'feature_92', 'feature_97', 'feature_98', 'feature_100', 'feature_103', 'feature_105', 'feature_106', 'feature_114', 'feature_117', 'feature_118', 'feature_119', 'feature_121', 'feature_127', 'feature_131', 'feature_132', 'feature_133', 'feature_137', 'feature_138', 'feature_139', 'feature_141', 'feature_143', 'feature_144', 'feature_147', 'feature_151', 'feature_152', 'feature_153', 'feature_154', 'feature_157', 'feature_158', 'feature_164', 'feature_166', 'feature_167', 'feature_168', 'feature_170', 'feature_172', 'feature_173', 'feature_178', 'feature_186', 'feature_189', 'feature_191', 'feature_194', 'feature_195', 'feature_197', 'feature_198', 'feature_199', 'feature_200', 'feature_201', 'feature_202', 'feature_205', 'feature_206', 'feature_208', 'feature_209', 'feature_212', 'feature_214', 'feature_218', 'feature_222', 'feature_223', 'feature_224', 'feature_230', 'feature_231', 'feature_234', 'feature_238', 'feature_239', 'feature_242', 'feature_244', 'feature_246', 'feature_247', 'feature_251', 'feature_253', 'feature_262', 'feature_264', 'feature_270', 'feature_273', 'feature_280', 'feature_283', 'feature_285', 'feature_296', 'feature_297', 'feature_305', 'feature_307', 'feature_308', 'feature_311', 'feature_314', 'feature_315', 'feature_318', 'feature_323', 'feature_325', 'feature_331', 'feature_332', 'feature_334', 'feature_335', 'feature_336', 'feature_340', 'feature_341', 'feature_342', 'feature_343', 'feature_345', 'feature_347', 'feature_349', 'feature_353', 'feature_356', 'feature_359', 'feature_361', 'feature_364', 'feature_365', 'feature_366', 'feature_369', 'feature_371', 'feature_378', 'feature_379', 'feature_381', 'feature_382', 'feature_383', 'feature_386', 'feature_388', 'feature_390', 'feature_392', 'feature_395', 'feature_406', 'feature_410', 'feature_421', 'feature_423', 'feature_424', 'feature_425', 'feature_428', 'feature_429', 'feature_434', 'feature_435', 'feature_443', 'feature_446', 'feature_449', 'feature_451', 'feature_452', 'feature_454', 'feature_456', 'feature_458', 'feature_460', 'feature_461', 'feature_462', 'feature_467', 'feature_470', 'feature_480', 'feature_481', 'feature_482', 'feature_491', 'feature_495', 'feature_pca0', 'feature_pca1', 'feature_pca2', 'feature_pca3', 'feature_pca4', 'feature_pca5', 'feature_pca6', 'feature_pca7', 'feature_pca9', 'feature_pca10', 'feature_pca11', 'feature_pca13', 'feature_pca14', 'feature_pca15', 'feature_pca16', 'feature_pca17', 'feature_pca18', 'feature_pca19', 'feature_pca20', 'feature_pca21', 'feature_pca22', 'feature_pca23', 'feature_pca24', 'column_0_mul_1', 'column_0_div_1', 'column_0_div_2', 'column_0_mul_3', 'column_0_div_3', 'column_0_mul_4', 'column_0_div_4', 'column_0_div_5', 'column_0_mul_6', 'column_0_div_6', 'column_0_mul_7', 'column_0_div_7', 'column_0_mul_8', 'column_0_div_8', 'column_0_mul_10', 'column_0_mul_11', 'column_0_div_11', 'column_0_div_14', 'column_0_div_16', 'column_0_div_17', 'column_0_mul_18', 'column_0_mul_20', 'column_0_div_20', 'column_0_mul_21', 'column_0_div_21', 'column_0_mul_23', 'column_0_div_23', 'column_0_mul_24', 'column_0_div_24', 'column_1_mul_3', 'column_1_div_3', 'column_1_mul_4', 'column_1_div_4', 'column_1_mul_5', 'column_1_div_5', 'column_1_mul_6', 'column_1_div_6', 'column_1_mul_8', 'column_1_div_8', 'column_1_mul_11', 'column_1_div_11', 'column_1_mul_16', 'column_1_div_16', 'column_1_mul_19', 'column_1_div_19', 'column_1_mul_20', 'column_1_div_20', 'column_1_mul_21', 'column_1_div_21', 'column_1_mul_23', 'column_1_div_23', 'column_1_mul_24', 'column_1_div_24', 'column_2_mul_3', 'column_2_div_3', 'column_2_mul_4', 'column_2_mul_5', 'column_2_mul_6', 'column_2_mul_8', 'column_2_div_8', 'column_2_mul_11', 'column_2_div_11', 'column_2_mul_16', 'column_2_div_16', 'column_2_mul_17', 'column_2_div_17', 'column_2_mul_19', 'column_2_mul_20', 'column_2_mul_21', 'column_2_mul_23', 'column_2_div_23', 'column_2_mul_24', 'column_3_mul_4', 'column_3_div_4', 'column_3_mul_5', 'column_3_div_5', 'column_3_mul_6', 'column_3_div_6', 'column_3_mul_7', 'column_3_div_7', 'column_3_mul_8', 'column_3_div_8', 'column_3_mul_10', 'column_3_mul_11', 'column_3_div_11', 'column_3_mul_12', 'column_3_mul_13', 'column_3_mul_14', 'column_3_mul_16', 'column_3_div_16', 'column_3_mul_17', 'column_3_div_17', 'column_3_mul_19', 'column_3_div_19', 'column_3_mul_20', 'column_3_div_20', 'column_3_mul_21', 'column_3_div_21', 'column_3_mul_22', 'column_3_div_22', 'column_3_mul_23', 'column_3_div_23', 'column_3_mul_24', 'column_3_div_24', 'column_4_mul_5', 'column_4_div_5', 'column_4_mul_6', 'column_4_div_6', 'column_4_mul_7', 'column_4_mul_8', 'column_4_div_8', 'column_4_mul_10', 'column_4_mul_11', 'column_4_div_11', 'column_4_mul_12', 'column_4_mul_16', 'column_4_div_16', 'column_4_mul_17', 'column_4_mul_19', 'column_4_mul_20', 'column_4_div_20', 'column_4_mul_21', 'column_4_div_21', 'column_4_mul_23', 'column_4_div_23', 'column_4_mul_24', 'column_5_mul_6', 'column_5_mul_7', 'column_5_mul_8', 'column_5_div_8', 'column_5_mul_10', 'column_5_mul_11', 'column_5_div_11', 'column_5_mul_12', 'column_5_mul_14', 'column_5_mul_16', 'column_5_div_16', 'column_5_mul_17', 'column_5_div_17', 'column_5_mul_19', 'column_5_mul_20', 'column_5_div_20', 'column_5_mul_21', 'column_5_div_21', 'column_5_mul_23', 'column_5_div_23', 'column_5_mul_24', 'column_5_div_24', 'column_6_mul_7', 'column_6_mul_8', 'column_6_div_8', 'column_6_mul_10', 'column_6_mul_11', 'column_6_div_11', 'column_6_mul_12', 'column_6_mul_14', 'column_6_mul_16', 'column_6_div_16', 'column_6_mul_17', 'column_6_mul_19', 'column_6_mul_20', 'column_6_div_20', 'column_6_mul_21', 'column_6_div_21', 'column_6_mul_22', 'column_6_mul_23', 'column_6_div_23', 'column_6_mul_24', 'column_7_mul_8', 'column_7_div_8', 'column_7_mul_11', 'column_7_div_11', 'column_7_mul_16', 'column_7_div_16', 'column_7_mul_19', 'column_7_div_19', 'column_7_mul_20', 'column_7_mul_21', 'column_7_mul_23', 'column_7_div_23', 'column_7_mul_24', 'column_8_mul_10', 'column_8_mul_11', 'column_8_div_11', 'column_8_mul_12', 'column_8_mul_13', 'column_8_div_13', 'column_8_mul_14', 'column_8_mul_16', 'column_8_div_16', 'column_8_mul_17', 'column_8_div_17', 'column_8_mul_19', 'column_8_div_19', 'column_8_mul_20', 'column_8_div_20', 'column_8_mul_21', 'column_8_div_21', 'column_8_mul_23', 'column_8_div_23', 'column_8_mul_24', 'column_8_div_24', 'column_9_mul_11', 'column_10_mul_11', 'column_10_div_11', 'column_10_mul_16', 'column_10_mul_23', 'column_10_mul_24', 'column_11_mul_12', 'column_11_div_12', 'column_11_mul_13', 'column_11_div_13', 'column_11_mul_14', 'column_11_div_14', 'column_11_mul_15', 'column_11_div_15', 'column_11_mul_16', 'column_11_div_16', 'column_11_mul_17', 'column_11_div_17', 'column_11_mul_18', 'column_11_div_18', 'column_11_mul_19', 'column_11_div_19', 'column_11_mul_20', 'column_11_div_20', 'column_11_mul_21', 'column_11_div_21', 'column_11_mul_22', 'column_11_div_22', 'column_11_mul_23', 'column_11_div_23', 'column_11_mul_24', 'column_11_div_24', 'column_12_mul_16', 'column_12_div_16', 'column_12_mul_23', 'column_12_mul_24', 'column_13_mul_16', 'column_13_div_16', 'column_13_mul_20', 'column_13_mul_21', 'column_13_mul_23', 'column_14_mul_16', 'column_14_mul_23', 'column_15_div_23', 'column_16_mul_17', 'column_16_div_17', 'column_16_mul_19', 'column_16_div_19', 'column_16_mul_20', 'column_16_div_20', 'column_16_mul_21', 'column_16_div_21', 'column_16_mul_22', 'column_16_mul_23', 'column_16_div_23', 'column_16_mul_24', 'column_16_div_24', 'column_17_mul_19', 'column_17_mul_20', 'column_17_mul_23', 'column_17_div_23', 'column_17_mul_24', 'column_17_div_24', 'column_18_div_23', 'column_19_mul_20', 'column_19_div_20', 'column_19_mul_21', 'column_19_div_21', 'column_19_mul_23', 'column_19_div_23', 'column_19_mul_24', 'column_19_div_24', 'column_20_mul_21', 'column_20_div_21', 'column_20_mul_22', 'column_20_div_22', 'column_20_mul_23', 'column_20_div_23', 'column_20_mul_24', 'column_20_div_24', 'column_21_mul_22', 'column_21_div_22', 'column_21_mul_23', 'column_21_div_23', 'column_21_mul_24', 'column_21_div_24', 'column_22_mul_23', 'column_23_mul_24', 'column_23_div_24']

In [9]:
pca = PCA(n_components=25)
X_pca = pca.fit_transform(df.drop(columns= ['id','target']))

In [10]:
new_features_df = pd.DataFrame(X_pca, columns=[f'feature_pca{i}' for i in range(X_pca.shape[1])])
new_features_df = new_features_df[[col for col in new_features_df.columns if col in generated]]
df = pd.concat([df, new_features_df], axis=1)

In [11]:
for col in df.select_dtypes(include=['int']):
    df[col] = pd.to_numeric(df[col], downcast='integer')
for col in df.select_dtypes(include=['float']):
    df[col] = pd.to_numeric(df[col], downcast='float')

In [12]:
gc.collect()

0

In [13]:
for i in tqdm(range(25)):
    for j in range(i + 1,25):
        if(f'column_{i}_mul_{j}' in generated):
            df[f'column_{i}_mul_{j}'] = (df.iloc[:, i+2] * df.iloc[:, j+2]).astype('float16')
        if(f'column_{i}_div_{j}' in generated):
            df[f'column_{i}_div_{j}'] = (df.iloc[:, i+2] / (df.iloc[:, j+2] + 1e-9)).astype('float16')
    
            if j % 10 == 0:
                gc.collect()

100%|██████████| 25/25 [00:13<00:00,  1.80it/s]


## Обучение 10 моделей стандартного catboost(подбор гиперпараметров оптуной не дал хороших результатов) на разных сидах

In [ ]:
 #Параметры
N_MODELS = 10  # Количество моделей в ансамбле
RANDOM_SEEDS = [42, 0, 123, 456, 789,152,777,1487,1321,65]  # Разные сиды для каждой модели
catboost_params = {
    'eval_metric': 'AUC',
    'random_seed': None,
    'verbose': 100
}

# Данные
X = df.drop(columns='target')
y = df['target']


# Обучение моделей и создание ансамбля
models = []
for i, seed in enumerate(RANDOM_SEEDS):
    print(f"Training model {i+1} with random seed {seed}...")
    catboost_params['random_seed'] = seed
    model = CatBoostClassifier(**catboost_params)
    model.fit(X, y)
    models.append(model)


In [ ]:

for i,model in enumerate(models):
    model.save_model(f'model_seed_{RANDOM_SEED[i]}.cbm')
    

In [ ]:
del X,y,df

gc.collect()

In [ ]:
test1 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-002/test/test_1.csv', usecols=test_features)
test2 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-002/test/test_2.csv', usecols=test_features)
test3 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-001/test/test_3.csv', usecols=test_features)
test4 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-001/test/test_4.csv', usecols=test_features)
test5 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-001/test/test_5.csv', usecols=test_features)
test6 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-002/test/test_6.csv', usecols=test_features)
test7 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-001/test/test_7.csv', usecols=test_features)
test8 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-001/test/test_8.csv', usecols=test_features)
test9 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-001/test/test_9.csv', usecols=test_features)
test10 = pd.read_csv('/kaggle/input/testdata/test-20241105T082246Z-001/test/test_10.csv', usecols=test_features)

In [ ]:
test = pd.concat([test1,test2,test3,test4,test5,test6,test7,test8,test9,test10], ignore_index=True)

In [ ]:
test_pca = pca.transform(test.drop(columns= ['id']))

In [ ]:
test_new_features_df = pd.DataFrame(X_pca, columns=[f'feature_pca{i}' for i in range(X_pca.shape[1])])

test_new_features_df = test_new_features_df[[col for col in test_new_features_df.columns if col in test_generate]]
test = pd.concat([test, test_new_features_df], axis=1)

In [ ]:
for i in tqdm(range(25)):
    for j in range(i + 1,25):
        if(f'column_{i}_mul_{j}' in generated):
            test[f'column_{i}_mul_{j}'] = (test.iloc[:, i+1] * test.iloc[:, j+1]).astype('float32')
        if(f'column_{i}_div_{j}' in generated):
            test[f'column_{i}_div_{j}'] = (test.iloc[:, i+1] / (test.iloc[:, j+1] + 1e-9)).astype('float32')
    
            if j % 10 == 0:
                gc.collect()

In [ ]:
test_data =test[test_generate]

In [ ]:
predictions = []
for model in models:
    test_preds = model.predict_proba(test)[:, 1]
    predictions.append(test_preds)
ensemble_preds = np.mean(predictions, axis=0)

In [ ]:
subm = pd.DataFrame({'id': test.id.tolist(), 'target': ensemble_preds})

In [ ]:
subm.to_csv('submission.csv',index = False)

In [ ]:
from IPython.display import FileLink
FileLink(r'submission.csv')